# Entraîner un modèle Transformer pour la traduction

**Objectif pédagogique :** Apprendre à construire et entraîner un modèle de traduction automatique Anglais → Français

## Installation et imports

**Objectif :** Installer les bibliothèques modernes pour le deep learning et le NLP

In [ ]:
# Installation des packages modernes (exécuter une seule fois)
# !pip install transformers datasets torch torchvision torchaudio
# !pip install pandas matplotlib tqdm scikit-learn

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

# Bibliothèques pour NLP moderne
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import math
import os
from typing import List
from timeit import default_timer as timer

## Configuration et détection matériel

**Concept important :** Il est crucial de détecter si un GPU est disponible pour accélérer l'entraînement !

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Device utilisé: {DEVICE}")

# Configuration de la reproductibilité
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    # Configuration pour performances reproductibles sur GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

## Hyperparamètres

In [ ]:
EMB_SIZE = 256  # Dimension des embeddings (doit être divisible par NHEAD)
NHEAD = 8       # Nombre de têtes d'attention (8 divise 256)
FFN_HID_DIM = 512  # Dimension de la couche feed-forward

# Adaptation de la batch size selon les capacités du device
BATCH_SIZE = 64 if DEVICE.type == 'cuda' else 32  # GPU peut traiter plus de données

NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
NUM_EPOCHS = 15  # Nombre d'époques d'entraînement
LEARNING_RATE = 0.0001  # Taux d'apprentissage conservateur

print(f"📊 Batch size: {BATCH_SIZE}")
print(f"🧠 Embedding size: {EMB_SIZE}")
print(f"👁️  Attention heads: {NHEAD}")
print(f"🔥 Epochs: {NUM_EPOCHS}")

## Chargement des données

**Dataset :** Nous utilisons un dataset de phrases parallèles Anglais-Français

In [ ]:
# Chargement du dataset (fourni)
try:
    import kagglehub
    path = kagglehub.dataset_download("devicharith/language-translation-englishfrench")
    csv_path = os.path.join(path, 'eng_-french.csv')
except:
    # Alternative: mettre votre chemin local
    csv_path = 'eng_-french.csv'

# Chargement avec pandas et sélection des colonnes nécessaires
df = pd.read_csv(
    csv_path,
    usecols=['English words/sentences', 'French words/sentences']
)
df = df.dropna()  # Supprimer les valeurs manquantes

print(f"📊 Dataset size: {len(df)} exemples")
# Affichage des premières lignes pour vérification
print(df.head())

## Tokenisation moderne

**Concept :** Les tokenizers pré-entraînés sont plus efficaces que construire le sien from scratch

Les tokenizers pré-entraînés sont construits à partir de corpus massifs et variés, ce qui leur permet de produire une segmentation plus cohérente et plus compacte que celle obtenue à partir de données limitées.
Cette segmentation réduit le nombre de tokens par texte, ce qui diminue le coût de calcul et améliore l’utilisation du contexte par le modèle.
Ils sont aussi parfaitement alignés avec les embeddings et les architectures existantes, garantissant une compatibilité et des performances optimales.
Construire son tokenizer from scratch introduit donc souvent une segmentation sous-optimale, de la dette technique et une perte de performance globale.

In [ ]:
# TODO: Initialisez les tokenizers pour anglais et français
# Aide: utilisez AutoTokenizer.from_pretrained() avec "bert-base-uncased" pour EN et "camembert-base" pour FR

from transformers import AutoTokenizer

tokenizer_en = # TODO: Tokenizer anglais
tokenizer_fr = # TODO: Tokenizer français

# Test des tokenizers
test_en = "Hello I love machine learning"
test_fr = "Bonjour j'adore l'apprentissage automatique"

print(f"🔤 Tokenization EN: {tokenizer_en.tokenize(test_en)}")
print(f"🔤 Tokenization FR: {tokenizer_fr.tokenize(test_fr)}")

print("✅ Tokenizers initialisés !")

## Construction du vocabulaire

In [ ]:
# Tokens spéciaux (fourni)
PAD_IDX = 0  # Padding
BOS_IDX = 1  # Beginning of Sentence
EOS_IDX = 2  # End of Sentence
UNK_IDX = 3  # Unknown word

def build_vocab(sentences, tokenizer, max_vocab=10000):
    """Construction du vocabulaire à partir des phrases"""
    word_count = {}

    # TODO: Parcourez toutes les phrases et comptez les mots
    for sentence in sentences:
        tokens = tokenizer.tokenize(str(sentence))
        for token in tokens:
            # TODO: Incrémentez le compteur pour ce token en utilisant word_count.get()
            word_count[token] = # TODO: Complétez

    # TODO: Triez les mots par fréquence (du plus fréquent au moins fréquent)
    # Utilisez la fonction sorted() avec une key appropriée
    sorted_words = # TODO: utilisez sorted() avec une key appropriée

    # Création du vocabulaire avec tokens spéciaux
    vocab = {"<pad>": PAD_IDX, "<bos>": BOS_IDX, "<eos>": EOS_IDX, "<unk>": UNK_IDX}

    # TODO: Ajoutez les mots les plus fréquents au vocabulaire
    for word, _ in sorted_words[:max_vocab-4]:  # -4 pour les tokens spéciaux
        vocab[word] = # TODO: Complétez avec la taille actuelle du vocabulaire

    return vocab

# Construction des vocabulaires
vocab_en = build_vocab(df['English words/sentences'], tokenizer_en)
vocab_fr = build_vocab(df['French words/sentences'], tokenizer_fr)

print(f"📖 Vocabulaire EN: {len(vocab_en)} mots")
print(f"📖 Vocabulaire FR: {len(vocab_fr)} mots")

## Dataset personnalisé

**Architecture :** PyTorch utilise des classes Dataset pour organiser les données

In [ ]:
def text_to_indices(text, tokenizer, vocab, max_len=50):
    """Convertit un texte en indices avec BOS et EOS"""
    tokens = tokenizer.tokenize(str(text))[:max_len-2]  # Place pour BOS/EOS

    # TODO: Convertissez les tokens en indices
    # Aide: utilisez vocab.get(token, UNK_IDX) pour gérer les mots inconnus
    indices = [BOS_IDX] +  [TODO: liste en compréhension] + [EOS_IDX]

    return torch.tensor(indices, dtype=torch.long)

class TranslationDataset(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)

    def __len__(self):
        # TODO: Retournez la taille du dataset
        return # TODO

    def __getitem__(self, idx):
        # TODO: Récupérez les textes anglais et français à partir de self.df
        en_text = # TODO
        fr_text = # TODO

        # TODO: Convertissez en indices en utilisant text_to_indices
        en_indices = # TODO
        fr_indices = # TODO

        return en_indices, fr_indices

print("✅ Classes Dataset définies !")

## DataLoaders

**Concept :** Le padding permet de traiter des phrases de longueurs différentes dans le même batch

In [ ]:
def collate_batch(batch):
    """Fonction pour créer des batchs avec padding"""
    src_batch, tgt_batch = zip(*batch)

    # TODO: Utilisez pad_sequence pour padding des séquences
    # Aide: padding_value=PAD_IDX, batch_first=True
    src_batch = # TODO
    tgt_batch = # TODO

    return src_batch, tgt_batch

# TODO: Divisez les données en train/test
# Aide: utilisez train_test_split avec test_size=0.1
train_df, test_df = # TODO

# TODO: Créez les datasets avec TranslationDataset
train_dataset = # TODO
test_dataset = # TODO

# TODO: Créez les DataLoaders
# Aide: batch_size=BATCH_SIZE, shuffle=True pour train, collate_fn=collate_batch
train_loader = DataLoader(# TODO)
test_loader = DataLoader(# TODO)

print(f"🔄 Training batches: {len(train_loader)}")
print(f"🔄 Test batches: {len(test_loader)}")

## Architecture du modèle Transformer

**Architecture :** Le Transformer utilise l'attention pour "encoder" les relations entre mots

In [ ]:
class PositionalEncoding(nn.Module):
    """Encodage positionnel pour que le modèle comprenne l'ordre des mots"""
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        # TODO: Créez le tensor d'encodage positionnel
        # uniquement des zéros au début
        pe = # TODO
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)

        # Formule mathématique pour l'encodage positionnel
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term) # Positions paires: sin
        pe[:, 1::2] = torch.cos(position * div_term) # Positions impaires: cos
        pe = pe.unsqueeze(0)

        self.register_buffer('pe', pe)

    def forward(self, x):
        # TODO: Ajoutez l'encodage positionnel aux embeddings
        # Attention: prendre seulement la longueur nécessaire
        x = x + # TODO
        return self.dropout(x)

print("✅ PositionalEncoding défini !")

In [ ]:
class TransformerTranslator(nn.Module):
    """Modèle Transformer complet pour la traduction"""
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=512, nhead=8,
                 num_encoder_layers=6, num_decoder_layers=6, dim_feedforward=2048, dropout=0.1):
        super().__init__()

        self.d_model = d_model

        # TODO: Créez les couches d'embedding pour source et target
        # Aide: nn.Embedding(vocab_size, d_model)
        self.src_embedding = # TODO
        self.tgt_embedding = # TODO

        # TODO: Ajoutez l'encodage positionnel avec PostionalEncoding
        self.pos_encoding = # TODO

        # TODO: Créez le Transformer
        # Aide: nn.Transformer avec tous les paramètres, batch_first=True
        self.transformer = # TODO

        # TODO: Couche de sortie pour prédire les mots
        # Aide: nn.Linear(d_model, tgt_vocab_size)
        self.output_projection = # TODO

        # Initialisation des poids (bonne pratique)
        self.init_weights()

    def init_weights(self):
        """Initialisation Xavier/Glorot pour meilleure convergence
        PyTorch utilise par défaut : Normal(0, 1) pour les embeddings
        PyTorch utilise : Uniform(-1/sqrt(in_features), 1/sqrt(in_features)) pour les Linear
        Les embeddings avec Normal(0,1) peuvent être trop grands Uniform(-0.1, 0.1) plus stable pour l'attention"""
        initrange = 0.1
        self.src_embedding.weight.data.uniform_(-initrange, initrange)
        self.tgt_embedding.weight.data.uniform_(-initrange, initrange)
        self.output_projection.bias.data.zero_()
        self.output_projection.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, tgt, src_mask=None, tgt_mask=None,
                src_key_padding_mask=None, tgt_key_padding_mask=None):

        # TODO: Appliquez les embeddings avec scaling et encodage positionnel
        # Aide: embedding * sqrt(d_model), puis pos_encoding
        src_emb = # TODO
        tgt_emb = # TODO

        # TODO: Passez dans le Transformer
        output = # TODO

        # TODO: Retournez la projection finale
        return # TODO

print("✅ Modèle Transformer défini !")

## Création des masques

**Concept crucial :** Les masques empêchent le modèle de "tricher" en regardant les mots futurs

In [ ]:
def create_masks(src, tgt):
    """Crée les masques nécessaires pour l'attention"""
    src_seq_len = src.shape[1]
    tgt_seq_len = tgt.shape[1]

    # Empêche le décodeur de voir les mots futurs
    tgt_mask = torch.triu(
        torch.ones(tgt_seq_len, tgt_seq_len, device=src.device) * float('-inf'),
        diagonal=1
    ).bool()

    # TODO: Créez les masques de padding
    # Aide: (src == PAD_IDX) pour identifier les tokens de padding
    src_padding_mask = # TODO
    tgt_padding_mask = # TODO

    return None, tgt_mask, src_padding_mask, tgt_padding_mask

print("✅ Fonction de masques définie !")

## Initialisation du modèle

In [ ]:
# TODO: Initialisez le modèle avec vos hyperparamètres
model = # TODO
# N'oubliez pas de mettre sur le bon device !

# TODO: Comptez les paramètres
total_params = sum(# TODO: utilisez p.numel() pour compter tous les paramètres)
trainable_params = sum(# TODO: pareil mais avec if p.requires_grad)

print(f"🧮 Paramètres totaux: {total_params:,}")
print(f"🧮 Paramètres entraînables: {trainable_params:,}")

# Estimation mémoire approximative
memory_mb = (total_params * 4) / (1024**2)  # 4 bytes par paramètre float32
print(f"💾 Mémoire approximative: {memory_mb:.1f} MB")

## Configuration de l'entraînement

**Optimisation :** Le gradient clipping évite les explosions de gradient

In [ ]:
# TODO: Définissez la fonction de coût
# Aide: nn.CrossEntropyLoss avec ignore_index=PAD_IDX
criterion = # TODO

# TODO: Définissez l'optimiseur
# Aide: torch.optim.AdamW avec lr=LEARNING_RATE betas=(0.9, 0.98), eps=1e-9 et weight_decay=0.01
optimizer = # TODO

# TODO: Ajoutez un scheduler
# Aide: torch.optim.lr_scheduler.StepLR avec step_size=5 et gamma=0.7
scheduler = # TODO

print("✅ Optimiseurs configurés !")

## Fonctions d'entraînement

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    """Entraîne le modèle pour une époque"""
    # TODO: Mettez le modèle en mode entraînement avec la méthode .train()
    model.# TODO

    total_loss = 0
    num_batches = len(train_loader)

    for src, tgt in tqdm(train_loader, desc="Training"):
        # TODO: Déplacez les données sur le bon device avec .to(device)
        src, tgt = # TODO

        # TODO: Préparez les données (input = tout sauf dernier, output = tout sauf premier)
        tgt_input = # TODO
        tgt_output = # TODO

        # TODO: Créez les masques avec create_masks
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = # TODO

        # TODO: Forward pass
        logits = model(# TODO: tous les arguments nécessaires)

        # TODO: Calculez la loss
        loss = criterion(# TODO: reshape approprié pour logits et tgt_output)

        # TODO: Backward pass avec gradient clipping
        # Reset gradients avec zero_grad()
        optimizer.#TODO
        # Calcul gradients avec .backward()
        loss.#TODO
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item()

    return total_loss / num_batches

print("✅ Fonction train_epoch définie !")

In [ ]:
def evaluate(model, test_loader, criterion, device):
    """Évalue le modèle sur les données de test"""
    # TODO: Mettez le modèle en mode évaluation
    model.# TODO

    total_loss = 0
    num_batches = len(test_loader)

    # TODO: Utilisez torch.no_grad() dans le with pour économiser la mémoire
    with # TODO:
        for src, tgt in tqdm(test_loader, desc="Evaluating"):

            src, tgt = src.to(device), tgt.to(device)

            tgt_input = tgt[:, :-1]
            tgt_output = tgt[:, 1:]

            src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_masks(src, tgt_input)

            logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask)
            loss = criterion(logits.reshape(-1, logits.shape[-1]), tgt_output.reshape(-1))

            total_loss += loss.item()

    return total_loss / num_batches

print("✅ Fonction evaluate définie !")

## Boucle d'entraînement principale

In [ ]:
# Listes pour stocker l'historique
train_losses = []
val_losses = []
best_val_loss = float('inf')

print("🚀 Début de l'entraînement !")
print("=" * 60)

for epoch in range(NUM_EPOCHS):
    start_time = timer()

    # TODO: Entraînement
    train_loss = train_epoch(# TODO: arguments)

    # TODO: Validation
    val_loss = evaluate(# TODO: arguments)

    scheduler.step()

    end_time = timer()

    # Sauvegarde de l'historique
    train_losses.append(train_loss)
    val_losses.append(val_loss)

    # Affichage des résultats
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}:")
    print(f"  📊 Train Loss: {train_loss:.4f}")
    print(f"  📊 Val Loss: {val_loss:.4f}")
    print(f"  ⏱️  Time: {end_time - start_time:.2f}s")
    print(f"  🎚️  LR: {scheduler.get_last_lr()[0]:.6f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
            'vocab_en': vocab_en,
            'vocab_fr': vocab_fr
        }, 'best_model_students.pt')
        print(f"  🎯 Nouveau meilleur modèle sauvegardé!")

    print("-" * 50)

print("✅ Entraînement terminé !")

## Visualisation des résultats

In [ ]:

plt.figure(figsize=(12, 8))

# Subplot 1: Courbes de loss
plt.subplot(2, 2, 1)
plt.plot(range(1, len(train_losses) + 1), train_losses,
         label='Train Loss', color='blue', linewidth=2)
plt.plot(range(1, len(val_losses) + 1), val_losses,
         label='Validation Loss', color='red', linewidth=2)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Évolution des losses pendant l\'entraînement')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 2: Différence train/val (détection overfitting)
plt.subplot(2, 2, 2)
diff_losses = [t - v for t, v in zip(train_losses, val_losses)]
plt.plot(range(1, len(diff_losses) + 1), diff_losses,
         color='orange', linewidth=2)
plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)
plt.xlabel('Epochs')
plt.ylabel('Train Loss - Val Loss')
plt.title('Écart Train/Validation (Overfitting)')
plt.grid(True, alpha=0.3)

# Subplot 3: Learning rate évolution
plt.subplot(2, 2, 3)
lrs = [LEARNING_RATE * (scheduler.gamma ** (i // scheduler.step_size))
       for i in range(len(train_losses))]
plt.plot(range(1, len(lrs) + 1), lrs, color='green', linewidth=2)
plt.xlabel('Epochs')
plt.ylabel('Learning Rate')
plt.title('Évolution du Learning Rate')
plt.yscale('log')  # Échelle logarithmique pour mieux voir
plt.grid(True, alpha=0.3)

# Subplot 4: Amélioration par époque
plt.subplot(2, 2, 4)
val_improvements = [0] + [val_losses[i-1] - val_losses[i]
                          for i in range(1, len(val_losses))]
colors = ['green' if x > 0 else 'red' for x in val_improvements]
plt.bar(range(1, len(val_improvements) + 1), val_improvements, color=colors, alpha=0.7)
plt.axhline(y=0, color='black', linestyle='-', alpha=0.5)
plt.xlabel('Epochs')
plt.ylabel('Amélioration Val Loss')
plt.title('Amélioration par Époque')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Sauvegarde du graphique
plt.savefig('training_analysis_correction.png', dpi=300, bbox_inches='tight')

# Analyse automatique des résultats
print("📈 Analyse automatique des résultats:")
print("=" * 50)
print(f"  🏆 Meilleure val loss: {best_val_loss:.4f}")
print(f"  📊 Loss finale train: {train_losses[-1]:.4f}")
print(f"  📊 Loss finale val: {val_losses[-1]:.4f}")
print(f"  📉 Réduction train loss: {((train_losses[0] - train_losses[-1]) / train_losses[0] * 100):.1f}%")
print(f"  📉 Réduction val loss: {((val_losses[0] - val_losses[-1]) / val_losses[0] * 100):.1f}%")

# Détection automatique des problèmes
final_gap = train_losses[-1] - val_losses[-1]
if final_gap > 0.5:
    print(f"  ⚠️  OVERFITTING détecté (écart: {final_gap:.3f})")
elif final_gap < -0.2:
    print(f"  ⚠️  UNDERFITTING possible (écart: {final_gap:.3f})")
else:
    print(f"  ✅ Équilibre train/val correct (écart: {final_gap:.3f})")

# Convergence
if len(val_losses) > 5:
    recent_std = np.std(val_losses[-5:])
    if recent_std < 0.01:
        print(f"  ✅ Convergence stable (std récente: {recent_std:.4f})")
    else:
        print(f"  📊 Convergence en cours (std récente: {recent_std:.4f})")

## Test du modèle

**Le moment de vérité :** Votre modèle sait-il traduire ?

In [ ]:
checkpoint = torch.load('best_model_students_correction.pt', map_location=DEVICE)
model.load_state_dict(checkpoint['model_state_dict'])
print(f"✅ Meilleur modèle chargé (époque {checkpoint['epoch']}, val_loss: {checkpoint['val_loss']:.4f})")

# Vocabulaire inversé pour la conversion
idx_to_word_fr = {idx: word for word, idx in vocab_fr.items()}

def translate_sentence(model, sentence, max_length=50, verbose=False):
    """Traduit une phrase anglaise en français avec décodage greedy"""
    model.eval()

    with torch.no_grad():
        # Préprocessing de la phrase source
        src_indices = text_to_indices(sentence, tokenizer_en, vocab_en)
        src = src_indices.unsqueeze(0).to(DEVICE)

        if verbose:
            print(f"Source tokens: {[vocab_en_inv.get(idx.item(), '<unk>') for idx in src_indices]}")

        # Initialisation avec BOS token
        tgt = torch.tensor([[BOS_IDX]], device=DEVICE)

        # Génération auto-régressive mot par mot
        for step in range(max_length):
            # Création des masques pour l'état actuel
            src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_masks(src, tgt)

            # Prédiction du prochain token
            logits = model(
                src, tgt,
                src_mask, tgt_mask,
                src_padding_mask, tgt_padding_mask
            )

            # Sélection du token le plus probable (greedy)
            next_token = logits[0, -1].argmax().unsqueeze(0).unsqueeze(0)

            if verbose:
                probs = F.softmax(logits[0, -1], dim=0)
                top_tokens = probs.topk(3)
                print(f"Step {step}: Top tokens: {[(idx_to_word_fr.get(idx.item(), '<unk>'), prob.item()) for idx, prob in zip(top_tokens.indices, top_tokens.values)]}")

            # Arrêt si EOS généré
            if next_token.item() == EOS_IDX:
                break

            # Ajout du nouveau token à la séquence
            tgt = torch.cat([tgt, next_token], dim=1)

        # Conversion des indices vers les mots
        translation = []
        for idx in tgt[0][1:]:  # Ignorer BOS au début
            word = idx_to_word_fr.get(idx.item(), '<unk>')
            if word in ['<eos>', '<pad>']:
                break
            translation.append(word)

        # Post-processing basique pour améliorer la lisibilité
        result = ' '.join(translation)
        # Corrections de ponctuation basiques
        result = result.replace(' ,', ',').replace(' .', '.').replace(' !', '!').replace(' ?', '?').replace('_', '')

        return result

print("✅ Fonction de traduction définie !")

# Vocabulaire inversé pour debug
vocab_en_inv = {idx: word for word, idx in vocab_en.items()}

In [ ]:
# Tests complets avec analyse de qualité

test_sentences = [
    "Hello, how are you?",
    "I love machine learning.",
    "The weather is beautiful today.",
    "Thank you very much.",
    "Good morning!",
    "What is your name?",
    "I am learning French.",
    "The book is on the table.",
    "Where are you going?",
    "This is a difficult problem."
]

# Traductions de référence pour comparaison
references = [
    "Bonjour, comment allez-vous ?",
    "J'adore l'apprentissage automatique.",
    "Le temps est beau aujourd'hui.",
    "Merci beaucoup.",
    "Bonjour !",
    "Comment vous appelez-vous ?",
    "J'apprends le français.",
    "Le livre est sur la table.",
    "Où allez-vous ?",
    "C'est un problème difficile."
]

print("🎯 Test de traduction avec évaluation:")
print("=" * 70)

total_score = 0
for i, sentence in enumerate(test_sentences):
    translation = translate_sentence(model, sentence)

    print(f"\n📝 Test {i+1}:")
    print(f"🇬🇧 EN: {sentence}")
    print(f"🤖 AI: {translation}")
    print(f"📚 REF: {references[i]}")

    # Évaluation qualitative simple
    prediction_words = set(translation.lower().split())
    reference_words = set(references[i].lower().split())

    if prediction_words and reference_words:
        overlap = len(prediction_words & reference_words)
        union = len(prediction_words | reference_words)
        jaccard = overlap / union if union > 0 else 0
        total_score += jaccard

        if jaccard > 0.6:
            quality = "🟢 Excellent"
        elif jaccard > 0.4:
            quality = "🟡 Correct"
        elif jaccard > 0.2:
            quality = "🟠 Partiel"
        else:
            quality = "🔴 Faible"

        print(f"📊 Similarité: {jaccard:.2f} {quality}")

    print("-" * 40)

average_score = total_score / len(test_sentences)
print(f"\n🏆 Score moyen: {average_score:.3f}")

if average_score > 0.5:
    print("🎉 Performance globale: BONNE !")
elif average_score > 0.3:
    print("👍 Performance globale: Correcte")
else:
    print("📚 Performance globale: À améliorer")

In [ ]:
# Test interactif

print("✨ Test interactif - Ajoutez vos propres phrases:")
print("=" * 50)

# Vos phrases personnalisées que vous pouvez modifier
custom_sentences = [
    "The cat is sleeping on the sofa.",
    "I want to travel to Paris.",
    "Can you help me please?",
    "The students are working hard.",
    "Technology is changing the world."
]

for i, sentence in enumerate(custom_sentences):
    print(f"\n🧪 Test personnalisé {i+1}:")
    translation = translate_sentence(model, sentence)
    print(f"🇬🇧 EN: {sentence}")
    print(f"🇫🇷 FR: {translation}")

    # Analyse détaillée pour 1 exemple
    if i == 0:
        print("\n🔍 Analyse détaillée:")
        detailed_translation = translate_sentence(model, sentence, verbose=True)